In [1]:
from keras import models
from keras import layers

Using TensorFlow backend.


In [3]:
from keras.datasets import boston_housing

In [4]:
(train_data, train_targets),(test_data, test_targets) = boston_housing.load_data()

57344/57026 [==============================] - 0s 2us/step


In [5]:
train_data.shape

(404, 13)

In [6]:
test_data.shape

(102, 13)

### Normalizing the data

In [7]:
mean = train_data.mean(axis = 0)
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std

In [8]:
test_data -= mean
test_data /= std

### building the model

In [12]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer  = 'rmsprop', loss = 'mse', metrics = ['mae'])
    return model

### K-fold validation

In [10]:
import numpy as np

In [14]:
k = 4
num_val_samples =  len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold # ',i)
    val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i+1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[: i * num_val_samples], 
                                         train_data[(i + 1) * num_val_samples:]], axis = 0)
    partial_train_targets = np.concatenate([train_targets[: i * num_val_samples], 
                                         train_targets[(i + 1) * num_val_samples:]], axis = 0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 1, verbose = 0)
    val_mse, val_mae = model.evaluate(val_data, val_targets,verbose = 0)
    all_scores.append(val_mae)

processing fold #  0
processing fold #  1
processing fold #  2
processing fold #  3


In [15]:
all_scores

[2.0969805009294262, 2.246963829097181, 2.874971269380928, 2.2850051539959293]

In [16]:
np.mean(all_scores)

2.375980188350866

In [20]:
k = 4
num_val_samples =  len(train_data) // k
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold # ',i)
    val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i+1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[: i * num_val_samples], 
                                         train_data[(i + 1) * num_val_samples:]], axis = 0)
    partial_train_targets = np.concatenate([train_targets[: i * num_val_samples], 
                                         train_targets[(i + 1) * num_val_samples:]], axis = 0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 1, verbose = 0)
    #val_mse, val_mae = model.evaluate(val_data, val_targets,verbose = 0)
    print(history.history.keys())
    mae_history = history.history['mean_absolute_error']
    all_mae_histories.append(mae_history)

processing fold #  0
dict_keys(['loss', 'mean_absolute_error'])
processing fold #  1
dict_keys(['loss', 'mean_absolute_error'])
processing fold #  2
dict_keys(['loss', 'mean_absolute_error'])
processing fold #  3
dict_keys(['loss', 'mean_absolute_error'])


In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mean_histories]) for in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)